In [9]:
! pip install 'git+https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git'

  Cloning https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-n3cw3qku
  Running command git clone -q https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-n3cw3qku
  Resolved https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to commit 10777d884085847e66e3a5da68c4bee53a3f855e
  Created wheel for JUSTFAIR-Tools: filename=JUSTFAIR_Tools-0.1.dev0-py3-none-any.whl size=1995 sha256=7a22726d9e2bf615aad44f11157ad962a23da84118bb0e3a56b6e2571f8d22ef
  Stored in directory: /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-ephem-wheel-cache-cfvf1tvw/wheels/9f/1e/47/4d44d5f699854fb0b65396a893e7d2e33825e8c788e91bd1ba
Successfully built JUSTFAIR-Tools


In [10]:
import JUSTFAIR_Tools as jt

In [12]:
dir(jt)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']